In [46]:
%matplotlib inline

import itertools
import os

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cosine, jaccard
import seaborn as sns
from pyclonal.parser import combineFiles
import glob
from plotly.offline import init_notebook_mode, iplot
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.html.widgets import interact
init_notebook_mode(connected=True)
from plotly.offline import iplot

In [59]:
#uses Ilyas code to get combined dataframe
df, seq_df = combineFiles(glob.glob('../sample_input_files/D*.tsv'))


../sample_input_files/D287.changeo.tsv looks like a changeo file
../sample_input_files/D280.changeo.tsv looks like a changeo file
../sample_input_files/D233.changeo.tsv looks like a changeo file
../sample_input_files/D299.changeo.tsv looks like a changeo file
../sample_input_files/D255.changeo.tsv looks like a changeo file


In [32]:
metadata=pd.read_csv('../sample_input_files/metadata_csv.txt',index_col='filename')

In [63]:
merged_df=df.join(metadata)

In [64]:
patient='D233'
filtered_df=merged_df[merged_df['patient_id']==patient] #change here
filtered_df = filtered_df.iloc[:, :-6]

In [65]:
res_df = {}
for l1, l2 in itertools.combinations(filtered_df.index, 2):
    res_df.setdefault(l2, {})[l1] = res_df.setdefault(l1, {})[l2] = 1 - cosine(
        list(filtered_df.loc[l1].values),
        list(filtered_df.loc[l2].values),
    )
        
cosine_data=pd.DataFrame(res_df).fillna(1)  #matrix with cosine differences

In [69]:
res_df = {}
for l1, l2 in itertools.combinations(filtered_df.index, 2):
    res_df.setdefault(l2, {})[l1] = res_df.setdefault(l1, {})[l2] =jaccard(
        list(filtered_df.loc[l1].values),
        list(filtered_df.loc[l2].values),
    )
        
jaccard_data=pd.DataFrame(res_df).fillna(1)  #matrix with jaccard differences

In [70]:
def plot_heatmap(method):
    METRIC = method
    if METRIC=='cosine':data=cosine_data
    if METRIC=='jaccard':data=jaccard_data
    
    
    fig, ax = plt.subplots(figsize=(12, 12))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    ax = sns.heatmap(data=data, cmap=cmap, square=True, linewidths=.5,
                 cbar_kws={"shrink": .5}, ax=ax)

In [71]:
interact(plot_heatmap, method=['jaccard', 'cosine'])

interactive(children=(Dropdown(description='method', options=('jaccard', 'cosine'), value='jaccard'), Output()…

<function __main__.plot_heatmap>